In [ ]:
# config = dotenv_values()

# host = config.get('ps_host')
# port = int(config.get('ps_port'))
# user = config.get('ps_username')
# password = config.get('ps_password')
# db = config.get('ps_database')

In [ ]:
import os
import mysql.connector
from dotenv import load_dotenv

load_dotenv()

# DB_HOST = os.getenv("ps_host")
# DB_USERNAME = os.getenv("ps_username")
# DB_PASSWORD = os.getenv("ps_password")
# DB_DATABASE = os.getenv("ps_database")

DB_HOST = os.getenv("ENDPOINT")
DB_USERNAME = os.getenv("USERNAME")
DB_PASSWORD = os.getenv("PASSWORD")
DB_DATABASE = os.getenv("DBNAME")


connection = mysql.connector.connect(
    host=DB_HOST,
    user=DB_USERNAME,
    password=DB_PASSWORD,
    database=DB_DATABASE,
    port=3306
    # ssl_verify_identity=True,
    # ssl_ca="path/to/ssl_cert"
)

print("Connected to the database")

In [ ]:
cursor = connection.cursor()
# Define the query
# query = "SELECT * FROM train"

# Execute the query in chunks of 100000 records
chunk_size = 10000
offset = 0
dfs = []
while True:
    # Execute the query
    cursor.execute(query + f" LIMIT {chunk_size} OFFSET {offset}")
    # Fetch the results
    results = cursor.fetchall()
    # If there are no more results, break out of the loop
    if not results:
        break
    # Convert the results to a DataFrame and append it to the list of DataFrames
    dfs.append(pd.DataFrame(results))
    # Increment the offset
    offset += chunk_size

# Concatenate all the DataFrames into a single DataFrame
df = pd.concat(dfs)

# Close the cursor and connection
cursor.close()
connection.close()


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
DB_HOST = os.getenv("ENDPOINT")
DB_USERNAME = os.getenv("USERNAME")
DB_PASSWORD = os.getenv("PASSWORD")
DB_DATABASE = os.getenv("DBNAME")
logging.info("dotenv loaded into config successfully")

Using `sqlalchemy` with
`create_engine(f'mysql+mysqldb://{user}:{password}@{host}/{db}')`

OperationalError: (1105, 'charset/name utf8mb3 is not supported')

The above exception was the direct cause of the following exception:

OperationalError                          Traceback (most recent call last)
/tmp/ipykernel_161486/2004886828.py in ()
      3 
      4 db = DBDataLoader()
----> 5 stores = db.load() # default stores table

OperationalError: (MySQLdb.OperationalError) (1105, 'charset/name utf8mb3 is not supported')
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
import connectorx as cx
conn="mysql://{user}:{pw}@{host}:{port}/{db}".format(user=DB_USERNAME,
                                                       pw=DB_PASSWORD,
                                                       host=DB_HOST,
                                                       db=DB_DATABASE,
                                                       port=3306,
                                                    #    ssl_verify_identity=True,
                                                    #    ssl_ca="path/to/ssl_cert"
                                                       )

In [ ]:
# load in data set using .sql file
query_file_path = '../src/scripts/train_store_hols.sql'

In [ ]:
query='''
select
	tr.`date`, tr.family, sum(tr.sales), sum(tr.onpromotion), 
	tr.store_nbr, st.city,
	YEAR(tr.`date`) `year`,
    MONTH(tr.`date`) `month`,
    DAY(tr.`date`) `day_of_month`,
	sum(transactions)
from
	train as tr
left join stores as st
    on tr.store_nbr = st.store_nbr
left join transactions txn 
    on tr.`date` = txn.`date` AND tr.store_nbr = txn.store_nbr
where
 	year(tr.`date`) between 2013 and 2015
group by tr.`date`, tr.family, st.city, tr.store_nbr
having st.city = 'Quito'
order by tr.`date`
'''


In [ ]:
with open(query_file_path, 'r') as query:
    chunks = db.load(query=query.read())
    # count += 1
    print(f'chunks size: {sys.getsizeof(chunks)}')
    logging.info(f"chunks loaded {sys.getsizeof(chunks)}")
    df = pd.DataFrame()
    for i in tqdm(range(sys.getsizeof(chunks)), desc='Reading from DB'):
        for chunk in chunks:
            df = pd.concat([df, chunk])

stmt='select * from full_df'
DF loaded confirm: 3000888 rows × 14 columns

In [ ]:
# stmt='select * from quito_frm_view'

chunks = db.load() # query=stmt

print(f'VwDump1 chunks size: {sys.getsizeof(chunks)}')
logging.info(f"VwDump1 chunks loaded {sys.getsizeof(chunks)}")

view_df = pd.DataFrame()
for i in tqdm(range(sys.getsizeof(chunks)), desc='Reading from View'):
    for chunk in chunks:
        view_df = pd.concat([view_df, chunk])

'SELECT * FROM VwDump1'
DF loaded confirm: 1972674 rows × 14 columns

In [ ]:
# import pandas as pd

# from ydata_profiling.utils.cache import cache_file
# from ydata_profiling import ProfileReport

# file_name = cache_file(
#     "pollution_us_2000_2016.csv",
#     "https://query.data.world/s/mz5ot3l4zrgvldncfgxu34nda45kvb",
# )

# df = pd.read_csv(file_name, index_col=[0])

# # Filtering time-series to profile a single site
# site = df[df["Site Num"] == 3003]

# # Setting what variables are time series
# type_schema = {
#     "NO2 Mean": "timeseries",
#     "NO2 1st Max Value": "timeseries",
#     "NO2 1st Max Hour": "timeseries",
#     "NO2 AQI": "timeseries",
#     "cos": "numeric",
#     "cat": "numeric",
# }

In [14]:
# https://stackoverflow.com/a/73319779

import polars as pl

days_data = pl.DataFrame({
    "weekday": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
    "date": ["1900-01-01", "1900-01-02", "1900-01-03", "1900-01-04", "1900-01-05"]
})

sales_data = pl.DataFrame({
    "sales": [1000, 20, 300, 40000, 555],
    "rep": ["Joe", "Mary", "Robert", "Thomas", "Susanna"],
    "date": ["1900-01-01", "1900-01-02", "1900-01-03", "1900-01-04", "1900-01-05"]
})

joined = sales_data.join(days_data, on="date")

# filtered = joined[lambda df: df["sales"] > 10000]

filtered = joined.filter(
                (pl.col("sales") > 10000)
            )

for row in filtered.iter_rows(named=True):
    print(f"Nice Job {row['rep']} and all this on a {row['weekday']}!")




Nice Job Thomas and all this on a Thursday!
